In [1]:
import  numpy  as  np
import libpysal as ps
from stwr.gwr import GWR, MGWR,STWR
from stwr.sel_bw import *
from stwr.utils import shift_colormap, truncate_colormap
import geopandas as gp
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from matplotlib.gridspec import GridSpec
import time
import csv 
import copy 

In [2]:
import rasterio
import rasterio.plot
import rasterio.features
import rasterio.warp
import pyproj

In [3]:
for i_curname in range(5):  
    str_filenames = "./Data_STWR/SimulatedData/case3/t{:d}/outfile_tol_adjt.csv".format(i_curname+1)
    #读入数据需要有这些
    cal_coords_list =[]
    cal_y_list =[]
    cal_X_list =[]
    delt_stwr_intervel =[0.0]
    csvFile = open(str_filenames, "r") 
    print("Current time stage is t{:d}".format(i_curname))
    
    df = pd.read_csv(csvFile,header = 0, names=['','cal_y','cal_x1','cal_x2','cal_coordsX','cal_coordsY','time_stamps'],
                     dtype = {"" : "float64","cal_y":"float64",
                              "cal_x1":"float64","cal_x2":"float64","cal_coordsX":"float64","cal_coordsY":"float64",
                              "time_stamps":"float64"},
                     skip_blank_lines = True,
                     keep_default_na = False)
    df.info()

    df = df.sort_values(by=['time_stamps'])  
    all_data = df.values
    tick_time = all_data[0,-1]
    cal_coord_tick = []
    cal_X_tick =[]
    cal_y_tick =[]
    time_tol = 1.0e-7
    lensdata = len(all_data)
    for row in range(lensdata):
        cur_time = all_data[row,-1]
        if(abs(cur_time-tick_time)>time_tol):
            cal_coords_list.append(np.asarray(cal_coord_tick))
            cal_X_list.append(np.asarray(cal_X_tick))
            cal_y_list.append(np.asarray(cal_y_tick))
            delt_t = cur_time - tick_time
            delt_stwr_intervel.append(delt_t) 
            tick_time =cur_time
            cal_coord_tick = []
            cal_X_tick =[]
            cal_y_tick =[]
        coords_tick = np.array([all_data[row,4],all_data[row,5]])
        cal_coord_tick.append(coords_tick)

        x_tick = np.array([all_data[row,2],all_data[row,3]])
        cal_X_tick.append(x_tick)
        y_tick = np.array([all_data[row,1]])
        cal_y_tick.append(y_tick)
    #最后在放一次
    #gwr解出最后一期 
    cal_cord_gwr = np.asarray(cal_coord_tick)
    cal_X_gwr  = np.asarray(cal_X_tick)
    cal_y_gwr = np.asarray(cal_y_tick)  
    cal_coords_list.append(np.asarray(cal_coord_tick))
    cal_X_list.append(np.asarray(cal_X_tick))
    cal_y_list.append(np.asarray(cal_y_tick))
    #stwr 
    stwr_selector_ = Sel_Spt_BW(cal_coords_list, cal_y_list, cal_X_list,#gwr_bw0,
                            delt_stwr_intervel)
    optalpha,optsita,opt_btticks,opt_gwr_bw0 = stwr_selector_.search() 
    stwr_model = STWR(cal_coords_list,cal_y_list,cal_X_list,delt_stwr_intervel,optsita,opt_gwr_bw0,tick_nums=opt_btticks+1,alpha =optalpha,recorded=1)
    stwr_results = stwr_model.fit()
    print(stwr_results.summary())
    
    gwr_selector = Sel_BW(cal_cord_gwr, cal_y_gwr, cal_X_gwr)
    gwr_bw= gwr_selector.search(bw_min=2)
    gwr_model = GWR(cal_cord_gwr, cal_y_gwr, cal_X_gwr, gwr_bw)
    gwr_results = gwr_model.fit()
    print(gwr_results.summary())
    gw_rscale = gwr_results.scale 
    gwr_residuals = gwr_results.resid_response
    

Current time stage is t0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
               67 non-null float64
cal_y          67 non-null float64
cal_x1         67 non-null float64
cal_x2         67 non-null float64
cal_coordsX    67 non-null float64
cal_coordsY    67 non-null float64
time_stamps    67 non-null float64
dtypes: float64(7)
memory usage: 3.7 KB
Model type                                                         Gaussian
Number of observations:                                                  67
Number of covariates:                                                     3

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                         760567.776
Log-likelihood:                                                    -407.863
AIC:                                                                821.725
AICc:                        